In [5]:
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler

In [2]:

import com.ibm.ibmos2spark.CloudObjectStorage

// @hidden_cell
var credentials = scala.collection.mutable.HashMap[String, String](
    "endPoint"->"https://s3.eu-geo.objectstorage.service.networklayer.com",
    "apiKey"->"r-gQWwP4FWfRO5vqz-YxRq9zzCSE3hkmi0R9UYqduLi0",
    "serviceId"->"iam-ServiceId-ddaaefe0-e7e7-4104-a120-68541de93756",
    "iamServiceEndpoint" -> "https://iam.eu-gb.bluemix.net/oidc/token")

var configurationName = "os_c0b4e5134f3343b4bfd5ba20c9e232ea_configs"
var cos = new CloudObjectStorage(sc, credentials, configurationName, "bluemix_cos")

import org.apache.spark.sql.SparkSession

val spark = SparkSession.
    builder().
    getOrCreate()
val taxiFile = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    option("inferSchema", "true").
    load(cos.url("wscoursedsproject-donotdelete-pr-yv8z4ynkhes74z", "2013_Green_Taxi_Trip_Data.csv"))
taxiFile.show(5)


[Stage 5:=============================>                             (1 + 1) / 2]+--------------------+--------------------+------------------+---------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+------------+------------+---------+------------------+------------------+------------------+---------+
|     pickup_datetime|    dropoff_datetime|Store_and_fwd_flag|rate_code|  Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|Total_amount|Payment_type|Trip_type|  Pickup_longitude|   Pickup_latitude| Dropoff_longitude|vendor_id|
+--------------------+--------------------+------------------+---------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+------------+------------+---------+------------------+------------------+------------------+---------+
|12/06/2013 12:11:...|12/06/2013 12:14:...|                 N|   

In [6]:
taxiFile.createOrReplaceTempView("taxifile")
val taxifence = spark.sql("""select Dropoff_latitude,Dropoff_longitude from taxifile where
Dropoff_latitude > 40.70 and
Dropoff_latitude < 40.86 and 
Dropoff_longitude > -74.02 and 
Dropoff_longitude < -73.93""")
taxifence.count

[Stage 7:=============================>                             (1 + 1) / 2]

589025

In [3]:
val assembler = (new VectorAssembler())
.setInputCols(Array("Dropoff_latitude", "Dropoff_longitude"))
.setOutputCol("features"))
val taxivector = assembler.transform(taxifence)
val taxifeat = taxivector.drop("Dropoff_latitude","Dropoff_longitude")

Name: Compile Error
Message: <console>:29: error: not found: type VectorAssembler
       val assembler = (new VectorAssembler()
                            ^
StackTrace: 

In [ ]:
val kmeans = new KMeans().setK(2).setSeed(1L)
val model = kmeans.fit(taxifeat)
println("Cluster Centers: ")
model.clusterCenters.foreach(println)
